In [28]:
#==task2 代码实践==
from sklearn.metrics.pairwise import cosine_similarity#余弦相似度
from scipy.stats import pearsonr#皮尔森相似度
import pandas as pd
import numpy as np

In [3]:
i = [1, 0, 0, 0]
j = [1, 0.5, 0.5, 0]
cosine_similarity([i,j])

array([[1.        , 0.81649658],
       [0.81649658, 1.        ]])

In [15]:
i = [1, 1,1,1]
j = [10, 10,10,10]
pearsonr(i,j)

G:\ph\Anaconda3\lib\site-packages\scipy\stats\stats.py:3010: RuntimeWarning: invalid value encountered in double_scalars
  r = r_num / r_den
G:\ph\Anaconda3\lib\site-packages\scipy\stats\stats.py:5256: RuntimeWarning: invalid value encountered in less
  x = np.where(x < 1.0, x, 1.0)  # if x > 1 then return 1.0


(nan, 1.0)

In [17]:
# userCF协同过滤，计算用户1 Alice 对商品E的打分

#==1== 做测试数据
def loadData(): 
    items = {
        'A':{1:5,2:3,3:4,4:3,5:1},
        'B': {1: 3, 2: 1, 3: 3, 4: 3, 5: 5}, 
        'C': {1: 4, 2: 2, 3: 4, 4: 1, 5: 5}, 
        'D': {1: 4, 2: 3, 3: 3, 4: 5, 5: 2}, 
        'E': {2: 3, 3: 5, 4: 4, 5: 1} 
    }
    users = {
        1:{'A':5,'B':3, 'C': 4, 'D': 4},
        2: {'A': 3, 'B': 1, 'C': 2, 'D': 3, 'E': 3}, 
        3: {'A': 4, 'B': 3, 'C': 4, 'D': 3, 'E': 5}, 
        4: {'A': 3, 'B': 3, 'C': 1, 'D': 5, 'E': 4}, 
        5: {'A': 1, 'B': 5, 'C': 5, 'D': 2, 'E': 1} 
    }
    return items,users
items,users = loadData()
print(items)
print(users)

{'A': {1: 5, 2: 3, 3: 4, 4: 3, 5: 1}, 'B': {1: 3, 2: 1, 3: 3, 4: 3, 5: 5}, 'C': {1: 4, 2: 2, 3: 4, 4: 1, 5: 5}, 'D': {1: 4, 2: 3, 3: 3, 4: 5, 5: 2}, 'E': {2: 3, 3: 5, 4: 4, 5: 1}}
{1: {'A': 5, 'B': 3, 'C': 4, 'D': 4}, 2: {'A': 3, 'B': 1, 'C': 2, 'D': 3, 'E': 3}, 3: {'A': 4, 'B': 3, 'C': 4, 'D': 3, 'E': 5}, 4: {'A': 3, 'B': 3, 'C': 1, 'D': 5, 'E': 4}, 5: {'A': 1, 'B': 5, 'C': 5, 'D': 2, 'E': 1}}


In [22]:
#创建DF
item_df = pd.DataFrame(items).T#.T行列转置
item_df
#ABCDE是物品
#1,2,3,4是用户

,1,2,3,4,5
A,5.0,3.0,4.0,3.0,1.0
B,3.0,1.0,3.0,3.0,5.0
C,4.0,2.0,4.0,1.0,5.0
D,4.0,3.0,3.0,5.0,2.0
E,NaN,3.0,5.0,4.0,1.0


In [25]:
user_df = pd.DataFrame(users).T
user_df
#ABCDE是物品
#1,2,3,4是用户
#此两个矩阵是同一批数据，行列角度不同。

,A,B,C,D,E
1,5.0,3.0,4.0,4.0,NaN
2,3.0,1.0,2.0,3.0,3.0
3,4.0,3.0,4.0,3.0,5.0
4,3.0,3.0,1.0,5.0,4.0
5,1.0,5.0,5.0,2.0,1.0


In [26]:
#userCF实现步骤：2、计算用户相似性矩阵，3、排序（得到前n个相似用户），4计算最终得分

In [29]:
#==2、计算用户相似矩阵==
similarity_matrix = pd.DataFrame(np.zeros((len(users),len(users))),index=[1,2,3,4,5],columns = [1,2,3,4,5])
similarity_matrix

,1,2,3,4,5
1,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0


In [50]:
#==2、计算用户相似矩阵==
#遍历上面users每条用户-物品评分数据
# {1: {'A': 5, 'B': 3, 'C': 4, 'D': 4}, 2: {'A': 3, 'B': 1, 'C': 2, 'D': 3, 'E': 3}, 3: {'A': 4, 'B': 3, 'C': 4, 'D': 3, 'E': 5}, 4: {'A': 3, 'B': 3, 'C': 1, 'D': 5, 'E': 4}, 5: {'A': 1, 'B': 5, 'C': 5, 'D': 2, 'E': 1}}
#items
#{'A': {1: 5, 2: 3, 3: 4, 4: 3, 5: 1}, 'B': {1: 3, 2: 1, 3: 3, 4: 3, 5: 5}, 'C': {1: 4, 2: 2, 3: 4, 4: 1, 5: 5}, 'D': {1: 4, 2: 3, 3: 3, 4: 5, 5: 2}, 'E': {2: 3, 3: 5, 4: 4, 5: 1}}
##此组数据是同一批数据，行列角度不同。
for userID in users:#【遍历的是字典的key】
   # print(userID)
    for otheruserId in users:
       # print(userID)
       # print(otheruserId)
        vec_user = []
        vec_otheruser = []
        if userID != otheruserId:#【同列排除，其它列，即其它用户】
            for itemId in items:#遍历字典的key
                itemRateList = items[itemId]#取出key对应的值
                #print("itemRateList",itemRateList)
                if userID in itemRateList and otheruserId in itemRateList:
                    #【in操作查询字典，看userID是否在字典的key里】
                    #【自己和其它用户都在列表里，说明两用户都对该物品有过评分】
                    vec_user.append(itemRateList[userID])#自己得分加入列表
                    vec_otheruser.append(itemRateList[otheruserId])#其它用户得分加入列表
                    
            #相似性矩阵（共现矩阵）np.corrcoef
            similarity_matrix[userID][otheruserId] = np.corrcoef(np.array(vec_user),np.array(vec_otheruser))[0][1]#相似性矩阵，取[0][1]，其它都是重复的。
            #自己id为列，其它用户id为行，把自己与其它用户的相似矩阵填入。
            print(similarity_matrix)

          1    2    3    4    5
1  0.000000  0.0  0.0  0.0  0.0
2  0.852803  0.0  0.0  0.0  0.0
3  0.000000  0.0  0.0  0.0  0.0
4  0.000000  0.0  0.0  0.0  0.0
5  0.000000  0.0  0.0  0.0  0.0
          1    2    3    4    5
1  0.000000  0.0  0.0  0.0  0.0
2  0.852803  0.0  0.0  0.0  0.0
3  0.707107  0.0  0.0  0.0  0.0
4  0.000000  0.0  0.0  0.0  0.0
5  0.000000  0.0  0.0  0.0  0.0
          1    2    3    4    5
1  0.000000  0.0  0.0  0.0  0.0
2  0.852803  0.0  0.0  0.0  0.0
3  0.707107  0.0  0.0  0.0  0.0
4  0.000000  0.0  0.0  0.0  0.0
5  0.000000  0.0  0.0  0.0  0.0
          1    2    3    4    5
1  0.000000  0.0  0.0  0.0  0.0
2  0.852803  0.0  0.0  0.0  0.0
3  0.707107  0.0  0.0  0.0  0.0
4  0.000000  0.0  0.0  0.0  0.0
5 -0.792118  0.0  0.0  0.0  0.0
          1         2    3    4    5
1  0.000000  0.852803  0.0  0.0  0.0
2  0.852803  0.000000  0.0  0.0  0.0
3  0.707107  0.000000  0.0  0.0  0.0
4  0.000000  0.000000  0.0  0.0  0.0
5 -0.792118  0.000000  0.0  0.0  0.0
          

In [46]:
# test = {1: 5, 2: 3, 3: 4, 4: 3, 5: 0}
# if 5 in test:
#     print(test[5])
# if 1 in test:
#     print(test[1])
# if 0 in test:
#     print(test[1])

0
5


In [61]:
print(similarity_matrix)

          1         2         3         4         5
1  0.000000  0.852803  0.707107  0.000000 -0.792118
2  0.852803  0.000000  0.467707  0.489956 -0.900149
3  0.707107  0.467707  0.000000 -0.161165 -0.466569
4  0.000000  0.489956 -0.161165  0.000000 -0.641503
5 -0.792118 -0.900149 -0.466569 -0.641503  0.000000


In [62]:
#==3、排序，（计算前n个相似用户）==
n = 2
#similarity_matrix[1] 取出一组即可，行与列同为5个用户，相互之间的相似性矩阵，其它都重复。
AsortUser = similarity_matrix[1].sort_values(ascending=False)#降序排列
print(AsortUser)
similarity_users = AsortUser[:n].index.tolist()#取前两个的index名转成列表，即[2,3]号用户
print(similarity_users)


2    0.852803
3    0.707107
4    0.000000
1    0.000000
5   -0.792118
Name: 1, dtype: float64
[2, 3]


In [65]:
#==4、计算用户1 Alice对商品5的打分==
#计算公式：Alice的平均打分 + （用户32与Alice相似度*（用户2给商品5的打分-用户2平均打分）+用户3与Alice相似度*（用户3给商品5的打分-用户3平均打分））/(用户2与Alice相似度+用户3与Alice的相似度)
# 4 + (0.85*(3-2.4)+0.7*(5-3.8))/(0.85+0.7) = 4.87
#基础分+权重分/总相似度
#user1 Alice的平均打分
base_score = np.mean(np.array([value for value in users[1].values()]))
# print(base_score)#4.0
weighted_scores = 0.
corr_values_sum = 0.
for user in similarity_users:#[3,2]
    corr_value = similarity_matrix[1][user]#取相似度，从相似矩阵similarity_matrix中取
    #打分平均值
    mean_user_score = np.mean(np.array([value for value in users[user].values()]))
    #加权分,分子
    weighted_scores += corr_value*(users[user]['E']-mean_user_score)
    #总相似度 分母，//下母，分母不能为0，不能被分divide
    corr_values_sum += corr_value
final_scores = base_score + weighted_scores/corr_values_sum

print("用户1 Alice对商品5的打分：",final_scores)#4.871979899370592

用户1 Alice对商品5的打分： 4.871979899370592


In [66]:
#补加到表中

user_df.loc[1]['E'] = final_scores
user_df

,A,B,C,D,E
1,5.0,3.0,4.0,4.0,4.87198
2,3.0,1.0,2.0,3.0,3.00000
3,4.0,3.0,4.0,3.0,5.00000
4,3.0,3.0,1.0,5.0,4.00000
5,1.0,5.0,5.0,2.0,1.00000


In [67]:
#==ItemCF 基于物品相似度的协同过滤 ，计算物品相似度（跟据共同用户的交互来计算相似程度），推荐==

In [68]:
#步骤：1、计算物品的相似性（5与其它1，2，3，4商品的相似性）；2、找出最相似的几个商品；3、根据Alice用户1对这几个相近的物品的打分计算对物品5的打分。

In [73]:
#和上面方法一样，只不过行列不同
#==1、计算物品相似矩阵==
similarity_matrix = pd.DataFrame(np.ones((len(items),len(items))),index=['A', 'B', 'C', 'D', 
'E'],columns = ['A', 'B', 'C', 'D', 'E'])#【【行列一样，用来计算商品两两相似性，用np.corrcoef】】
for itemId in items:
    for otheritemId in items:
        vec_item = []
        vec_otheritem = []
        if itemId != otheritemId:
            for userId in users:
                userRateList = users[userId]
                if itemId in userRateList and otheritemId in userRateList:
                    #用户对这两个商品都评过分。都评过的计算相似度
                    vec_item.append(userRateList[itemId])#本商品
                    vec_otheritem.append(userRateList[otheritemId])#其它商品
                    
            #算自己与其它商品的相似矩阵
            similarity_matrix[itemId][otheritemId] = np.corrcoef(np.array(vec_item),np.array(vec_otheritem))[0][1]
            
print(similarity_matrix)     
            

          A         B         C         D         E
A  1.000000 -0.476731 -0.123091  0.532181  0.969458
B -0.476731  1.000000  0.645497 -0.310087 -0.478091
C -0.123091  0.645497  1.000000 -0.720577 -0.427618
D  0.532181 -0.310087 -0.720577  1.000000  0.581675
E  0.969458 -0.478091 -0.427618  0.581675  1.000000


In [77]:
#==2、排度，查前n个相似商品==
n = 3
similarity_items = similarity_matrix['E'].sort_values(ascending = False)[:n].index.tolist()
similarity_items = [item for item in similarity_items if item !='E']#去掉自己
similarity_items

['A', 'D']

In [78]:
#==3、计算打分==
#公式同上
base_score = np.mean(np.array([value for value in items['E'].values()]))#E的平均打分
weighted_scores = 0. #权重分
corr_values_sum = 0. #相拟度
for item in similarity_items:
    corr_value = similarity_matrix['E'][item]
    mean_item_score = np.mean(np.array([value for value in items[item].values()]))#相似商品自身平均打分
    weighted_scores += corr_value*(users[1][item]-mean_item_score)
    corr_values_sum += corr_value
final_scores = base_score + weighted_scores/corr_values_sum
print("用户1 Alice对商品5的打分：",final_scores)#4.6


用户1 Alice对商品5的打分： 4.6


In [79]:
user_df.loc[1]['E'] = final_scores 
user_df 

,A,B,C,D,E
1,5.0,3.0,4.0,4.0,4.6
2,3.0,1.0,2.0,3.0,3.0
3,4.0,3.0,4.0,3.0,5.0
4,3.0,3.0,1.0,5.0,4.0
5,1.0,5.0,5.0,2.0,1.0


In [ ]:
#基于物品相似性推荐itemCF的是4.6分，基于用户相似推荐userCF的是4.87分
#差不太多